In [1]:
import numpy as np
import pandas as pd
import requests
import regex as re
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from xmltodict import parse
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

2022-05-16 14:39:09.318172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kaygun/local/lib
2022-05-16 14:39:09.318201: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

In [2]:
def getSubjectGuardian(subject):
    with requests.get(f'https://www.theguardian.com/{subject}/rss') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

def getText(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

def getSummary(text,k):
    sentences = sent_tokenize(text)
    N = len(sentences)
    vectorizer = CountVectorizer()
    matrix = vectorizer.fit_transform(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(N),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

def getKeywords(text,sw,k):
    sentences = sent_tokenize(text)    
    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(sentences)
    words = vectorizer.get_feature_names_out()
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]
    return sorted(zip(weights,words),key=lambda x: x[0], reverse=True)[:k]

In [3]:
economy = getSubjectGuardian('economy')
text = getText(economy[0]['link'])
getSummary(text,4)

[(3.2008312673793955,
  36,
  '“The humanitarian crisis caused by the war in Ukraine, and the precipitating unpredictable operating environment, have led McDonald’s to conclude that continued ownership of the business in Russia is no longer tenable.” UK diesel prices have hit record highs, pushing up the costs faced by some firms and motorists.'),
 (14.566277780219078,
  45,
  '“The average price of petrol is also on the rise having gone up nearly 3p a litre since the start of the month to 166.65p which means it’s less than a penny away from the all-time high of 167.30p set on 22 March.” NEWUK diesel price at all time record of 180p a litre… UK had been dependent on Russia for a third of imports.Petrol price just below (less than 1p a litre) all time record at 166.65p… this is all after the effect of 5p duty cut.Source: @TheRAC_UK Here are next year’s growth forecast for the EU: Growth forecast for 2023 (%):🇮🇪 4.4🇲🇹 4.0🇸🇰 3.6🇷🇴 3.6🇨🇾 3.5🇪🇸 3.4🇸🇮 3.1🇧🇬 3.1🇬🇷 3.1🇭🇷 3.0🇵🇱 3.0🇱🇻 2.9🇨🇿 2.7🇵

In [4]:
swEN = stopwords.words('english')
getKeywords(text,swEN,10)

[(3.7542756026490696, 'growth'),
 (3.671468847517566, 'forecast'),
 (2.6621427633662873, 'eu'),
 (2.4205154535443976, 'record'),
 (2.4120368731668984, 'price'),
 (2.240888627547999, 'war'),
 (2.196063202166399, 'litre'),
 (2.1347569159921798, 'time'),
 (1.9581011915500874, 'year'),
 (1.5570021602165718, 'inflation')]

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

In [5]:
NER = spacy.load("en_core_web_sm")

def getProperNames(URL):
    res = NER(getText(URL))
    return [(word.text, word.label_) for word in res.ents if word.label_ in ['ORG','PERSON','GPE','NORP','LOC']]

In [7]:
getProperNames(economy[0]['link'])

[('The European Commission', 'ORG'),
 ('European', 'NORP'),
 ('Ukraine', 'GPE'),
 ('EC', 'ORG'),
 ('EU', 'ORG'),
 ('EU', 'ORG'),
 ('Forecast', 'ORG'),
 ('Commission', 'ORG'),
 ('the European Central Bank’s', 'ORG'),
 ('Commission', 'ORG'),
 ('EU', 'ORG'),
 ('Russia', 'GPE'),
 ('Ukraine', 'GPE'),
 ('EU', 'ORG'),
 ('EU', 'ORG'),
 ('Russian', 'NORP'),
 ('Navigator Terminals', 'ORG'),
 ('Essex', 'GPE'),
 ('Russian', 'NORP'),
 ('Vladimir Putin', 'PERSON'),
 ('Essex Police', 'ORG'),
 ('PA Media', 'ORG'),
 ('Russian', 'NORP'),
 ('📣', 'ORG'),
 ('Russian', 'NORP'),
 ('Essex', 'ORG'),
 ('UK', 'GPE'),
 ('Putin', 'PERSON'),
 ('Ukraine', 'GPE'),
 ('Ukraine', 'GPE'),
 ('Russian', 'NORP'),
 ('UK', 'GPE'),
 ('Russian', 'NORP'),
 ('👇https://t.co/q5p6kqVCgG Greenpeace', 'ORG'),
 ('Georgia Whitaker', 'PERSON'),
 ('Greenpeace UK', 'ORG'),
 ('UK', 'GPE'),
 ('Putin', 'PERSON'),
 ('Ukraine', 'GPE'),
 ('UK', 'GPE'),
 ('Russian', 'NORP'),
 ('Putin', 'PERSON'),
 ('UK', 'GPE'),
 ('Russian', 'NORP'),
 ('Russian',

### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

In [53]:
analyzer = SentimentIntensityAnalyzer()

def extremeSentences(text):
    sentences = sent_tokenize(text)
    scores = [analyzer.polarity_scores(s) for s in sentences]
    res = list(zip(sentences,scores))
    mostNeg = sorted(res, key = lambda x: x[1]['neg'], reverse=True)[0][0]
    mostPos = sorted(res, key = lambda x: x[1]['pos'], reverse=True)[0][0]
    return (mostPos,mostNeg)

In [54]:
extremeSentences(getText(economy[3]['link']))

('Bonus payouts for Greggs’ outgoing chief executive, Roger Whiteside, amounted to more than double his basic salary of £575,209, taking his total package to £1.9m including benefits.',
 'The investment adviser Pirc said shareholders should vote against the bakery chain’s remuneration report, arguing that Whiteside’s pay was excessive and amounted to 79 times that of a regular employee.')

In [52]:
film = getSubjectGuardian('film')
extremeSentences(getText(film[1]['link']))

('She tells no one.',
 'She’s devoted to her mother and sister but dreams of emigrating to the United States with her poetry-reading love interest, Murdo (Will Fletcher).')